In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline
'''import libraries used, json to read the data, 
randint for randomly generated paragraphs, 
and pandas to create a dataframe to be written to csv'''
import pandas as pd
from tqdm import tqdm
import json
import csv
fname = 'dev-SQuAD-v2.0'
f = open(fname + '.json')
#load in data as python dictionary using the json library
data = json.load(f)
data = data["data"]
samples = pd.read_csv("Negation_dev.csv")
originals = pd.read_csv("dev_start.csv")
originals = originals.drop('New_Context', axis=1)
originals.drop(originals.columns[originals.columns.str.contains(
    'unnamed', case=False)], axis=1, inplace=True)
electra = pipeline("question-answering", model="deepset/electra-base-squad2")

In [3]:
try:
    final = pd.read_csv("Task13P-U-dev.csv").to_dict('records')
except:
    final = []

In [ ]:
address_output = "Task13P-U-dev.tsv"
record_file = open(address_output, "w")  ## this is 'w' = write mode
tsv_writer_temp = csv.writer(record_file, delimiter='\t', lineterminator='\n')
tsv_writer_temp.writerow(['Question_Id', 'Context', 'Original_Question', 'Original_Answer', 'Modified_Question', 'O-Q model2 answer span', 'O-Q model2 start', 'O-Q model2 end', 'O-Q model2 score', 'O-Q model4 answer span', 'O-Q model4 start', 'O-Q model4 end', 'O-Q model4 score', 'O-Q electra answer span', 'O-Q electra start', 'O-Q electra end', 'O-Q electra score', 'O-Q roberta answer span', 'O-Q roberta start', 'O-Q roberta end', 'O-Q roberta score', 'O-Q albert score', 'O-Q albert answer span', 'O-Q sgnet answer span', 'O-Q retro answer span', 'P-U model2 answer span', 'P-U model2 start', 'P-U model2 end', 'P-U model2 score', 'P-U model4 answer span', 'P-U model4 start', 'P-U model4 end', 'P-U model4 score', 'P-U electra answer span', 'P-U electra start', 'P-U electra end', 'P-U electra score', 'P-U roberta answer span', 'P-U roberta start', 'P-U roberta end', 'P-U roberta score', 'P-U albert score'])
record_file.close()

In [4]:
#odata = json.load(open("SGO.json"))
#odata = [json.load(open("albert0.json")), json.load(open("SGO.json")), json.load(open("retrospective.json"))]
def get_output_models(id, question, context):
    row = {}
    
    preds = electra(question=question, context=context,)
    row["P-U electra answer span"] = preds["answer"]
    row["P-U electra start"] = preds["start"]
    row["P-U electra end"] = preds["end"]
    row["P-U electra score"] = preds["score"]
    #row["P-U " + labels[i + 1] + " answer span"] = odata[id] 
    #for j in range(0, len(odata)):
#        row["P-U " + labels[i + j] + " answer span"] = odata[j][id]
    return row

In [5]:
def save_csv(df):
    final_df = pd.DataFrame(final)
    final_df.to_csv("Task13P-U-dev.csv")

def append_row(row):
    with open("Task13P-U-dev.tsv",'a', encoding='utf-8') as f1: ## this is 'a' = apend mode
        writer=csv.writer(f1, delimiter='\t',lineterminator='\n',)
        nrow = []
        for k, v in row.items():
            nrow.append(v)
        writer.writerow(nrow) # These are the value for the new row in df 
    

In [ ]:
#this is the dataframe that will be converted to pandas and then to csv
#final = []
#counter = 0
#loop through all different data titles/topics associated!
for i in tqdm(range(len(data)), desc="Different topics: "):
    #get number of paragraphs per topic
    length = len(data[i]['paragraphs'])
    #get list of all paragraphs for later use
    paragraphs = data[i]['paragraphs']
    #loop through each paragraph
    for j in tqdm(range(len(paragraphs)), leave=False, desc="Going through paragraphs: "):
        #get all of the questions associated per paragraph
        questions = paragraphs[j]['qas']
        #get the context paragraph value
        context = paragraphs[j]['context']
        #loop through all questions in each topic paragraph
        for k in range(0, len(questions)):
            if(questions[k]['is_impossible'] == False):
                #get the question id
                id = questions[k]['id']
                if not any(d['Question_Id'] == id for d in final):
                    #get question value
                    question = questions[k]['question']
                    answer = questions[k]["answers"][0]["text"]
                    relevant_pot_qs = samples.loc[samples['Question_Id'] == id]
                    for index, row in relevant_pot_qs.iterrows():
                        frow = originals.loc[originals['Question_Id'] == id]
                        final_row = frow.to_dict('records')[0]
                        #final_row = {"Question_ID": id,  "Context": context, "Original_Question": question, "Original_Answer": answer, "Modified_Question": row["Modified_Question"]}
                        #for k, v in get_og_ans(id).items():
                        #    final_row[k] = v
                        for k, v in get_output_models(id, str(row["Modified_Question"]), context).items():
                            final_row[k] = v
                        #print(final_row)
                        final.append(final_row)
                        append_row(final_row)
                        save_csv(final)
                else:
                    used_id = [d for d in final if d.get('Question_Id') == id]
                    length = len(used_id)
                    used_originals = originals.loc[originals['Question_Id'] == id]
                    olen = used_originals.shape[0]
                    if(length < olen):
                        final = final.loc[final['Question_Id'] != id]
                        #get question value
                        question = questions[k]['question']
                        answer = questions[k]["answers"][0]["text"]
                        relevant_pot_qs = samples.loc[samples['Question_Id'] == id]
                        for index, row in relevant_pot_qs.iterrows():
                            frow = originals.loc[originals['Question_Id'] == id]
                            final_row = frow.to_dict('records')[0]
                            #final_row = {"Question_ID": id,  "Context": context, "Original_Question": question, "Original_Answer": answer, "Modified_Question": row["Modified_Question"]}
                            #for k, v in get_og_ans(id).items():
                            #    final_row[k] = v
                            for k, v in get_output_models(id, str(row["Modified_Question"]), context).items():
                                final_row[k] = v
                            #print(final_row)
                            final.append(final_row)
                            append_row(final_row)
                            save_csv(final)
                        
                    

            

Going through paragraphs:  70%|███████████████████████████████████████▍                | 31/44 [05:18<05:15, 24.25s/it]

In [ ]:
final_df = pd.DataFrame(final)

In [ ]:
final_df.head()

In [ ]:
final_df.to_csv("Task13P-U-dev.csv")